In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from Modules import step_3
from scipy.stats import gaussian_kde
from PIL import Image
import warnings
warnings.filterwarnings('ignore')

In [5]:
years_to_load = [2016, 2017]  
dataframes = step_3.load_dataframes(years_to_load)

In [1]:
def ajouter_côté_équipe_attaquante(df):
    
    """Si ce n"est pas fait, Crée une nouvelle colonne pour indiquer si l'équipe attaquante est à gauche ou à 
        droite"""

    df[['CoordonnéesX', 'CoordonnéesY']] = df['Coordinates'].apply(lambda x: pd.Series(eval(x)))

    df['CôtéÉquipeAttaquante'] = np.where(df['CoordonnéesX'] > 0, 'left', 'right')
    #df['CôtéÉquipeAttaquante'] = np.where(df['CoordonnéesX'] < 0, 'left', 'right')

    return df

In [ ]:
def obtenir_coordonnées_côté_gauche(df):
    """Pour les tirs du côté droit, obtenez les coordonnées correspondantes du côté gauche."""
    
    # Identify which shots are on the left and right
    cote_gauche = df['CôtéÉquipeAttaquante'] == 'left'
    cote_droit = df['CôtéÉquipeAttaquante'] == 'right'

    # Keep original coordinates for left-side shots and negate for right-side shots
    df['CoordonnéesX_G'] = np.where(cote_gauche, df['CoordonnéesX'], -df['CoordonnéesX'])
    df['CoordonnéesY_G'] = np.where(cote_gauche, df['CoordonnéesY'], -df['CoordonnéesY'])

    return df

In [ ]:
# Configure path
save_fig = "data/Shotmap_interactive.html"

def préparer_données_de_cartedetir(df, largeur_bande_kernel=1, grid_size=(101, 101), echelle = 100):
    """
    Produit un DataFrame prêt à être utilisé pour la création de la visualisation de la carte de tir
    
    Paramètres: 
        largeur_bande_kernel: largeur de la bande pour l'estimation du noyau gaussien 2D
        grid_size: taille de la grille discrète pour laquelle nous calculons les densités de probabilité estimées par le noyau
    """
    
    df_copie = df.copy()
    # Check initial data size
    print(f"Initial combined_df size: {df_copie.shape}")
    
    # Remove rows with missing coordinates
    df_copie = df_copie[~df_copie[['CoordonnéesX', 'CoordonnéesY']].isnull().any(axis=1)]
    print(f"After removing null coordinates: {df_copie.shape}")
    
    df_copie = df_copie[(df_copie['ID'].astype(str).str.slice(4, 6) == '02') & (df_copie['Saison'] <= 2020)]
    
    # Filter out playoffs
    df_copie = df_copie[df_copie['ID'].astype(str).str.slice(4, 6) == '02']
    print(f"After filtering for regular season: {df_copie.shape}")
    
    # Invert coordinates for right-side shots
    df_copie = obtenir_coordonnées_côté_gauchee(df_copie)
    print(f"After adjusting shot coordinates: {df_copie.shape}")
    
    # Filter out shots from the other side of the red line
    df_copie = df_copie[df_copie['CoordonnéesX_G'] > 0]
    print(f"After filtering for shots beyond red line: {df_copie.shape}")
    
    df_copie = df_copie[~df_copie[['CoordonnéesX', 'CoordonnéesY']].isnull().any(axis=1)]
    
    # Filtrer les séries éliminatoires : nous examinons les statistiques agrégées
    df_copie = df_copie[df_copie['ID'].astype(str).str.slice(4, 6) == '02']  # `GameID` is now `ID`
    
    # Si l'équipe attaquante est du côté droit, inversez les coordonnées pour les avoir toutes du même côté
    df_copie = obtenir_coordonnées_côté_gauchee(df_copie)
    
    # Filtrer les tirs qui se produisent de l'autre côté de la ligne rouge
    ###df_copie = df_copie[df_copie['CoordonnéesX_G'] > 0]
    #df_copie = df_copie[df_copie['CoordonnéesX_G'] >= 0] 
    
    # Configuration pour l'estimation de la densité du noyau
    output_df = pd.DataFrame()
    y = np.linspace(0, 89, grid_size[0])
    x = np.linspace(-42.5, 42.5, grid_size[1])
    xy = np.array(np.meshgrid(x, y)).reshape(2, -1)
    
    # Nombre de matchs joués par équipe par saison
    nombre_de_matchs_par_équipe = { 
        2016 : 82,
        2017 : 82,
        2018 : 82,
        2019 : 70,
        2020 : 56
    }
    
    # Calculer la densité du noyau pour chaque saison et chaque équipe
    for saison in sorted(list(set(df_copie['Saison']))):
        
        df_saison = df_copie[df_copie['Saison'] == saison]
        xy_coords_saison = df_saison[['CoordonnéesY_G', 'CoordonnéesX_G']].to_numpy().T
        noyau_saison = gaussian_kde(xy_coords_saison, bw_method=largeur_bande_kernel)
        densité_probabilité_grille_saison = noyau_saison(xy)  # Densité par pied carré par tir
        densité_probabilité_grille_saison = densité_probabilité_grille_saison * echelle * (
            len(df_saison) / (len(set(df_saison['Team'])) * nombre_de_matchs_par_équipe[saison]))  # Mise à l'échelle
        
        for équipe in sorted(list(set(df_saison['Team']))):  # `ÉquipeAttaquante` is now `Team`
            
            df_équipe_saison = df_saison[df_saison['Team'] == équipe]
            noyau_équipe_saison = gaussian_kde(df_équipe_saison[['CoordonnéesY_G', 'CoordonnéesX_G']].to_numpy().T, bw_method=largeur_bande_kernel)
            densité_probabilité_grille_équipe_saison = noyau_équipe_saison(xy)  # Densité par pied carré par tir
            densité_probabilité_grille_équipe_saison = densité_probabilité_grille_équipe_saison * echelle * (len(df_équipe_saison) / nombre_de_matchs_par_équipe[saison])  # Mise à l'échelle
            différentiel_équipe_saison = densité_probabilité_grille_équipe_saison - densité_probabilité_grille_saison

            output_df[f"{saison} {équipe}"] = différentiel_équipe_saison
            
        print(f"Saison {saison} - Terminée")
                
    return output_df

def générer_carte_tirs(carte_tirs_df, image_patinoire, grid_size=(101, 101), echelle=100):
    """
    Produire une visualisation de la carte des tirs
    
    Paramètres :
        carte_tirs_df : sortie de la fonction préparer_carte_tirs()
        image_patinoire : image fournie de la patinoire de hockey
        grid_size : taille de la grille discrète pour laquelle nous calculons les densités de probabilité estimées par le noyau
    """
    
    image_patinoire = image_patinoire.crop((image_patinoire.size[0]/2, 0, image_patinoire.size[0], image_patinoire.size[1]))
    image_patinoire = image_patinoire.transpose(Image.Transpose.ROTATE_90)

    y = np.linspace(0, 89, grid_size[0])
    x = np.linspace(-42.5, 42.5, grid_size[1])
    z = np.array(carte_tirs_df[carte_tirs_df.columns[0]]).reshape(grid_size[1], grid_size[0])
    fig = go.Figure()

    fig.add_trace(
        go.Contour(
            z=z,
            x=x,
            y=y,
            hoverongaps=False,
            opacity=0.8,
            zmin=-0.55, zmax=0.55,
            colorscale=[[0, '#0000FF'], [0.5, '#FFFFFF'], [1, '#FF0000']],
            colorbar=dict(title=f"Tirs excédentaires par heure par {echelle} pieds carrés", titleside="right")
        )
    )

    menus_de_mise_à_jour = [
        {
            'buttons': 
               [{'method': 'restyle', 'label': col,'args': [{'z': [np.array(carte_tirs_df[col]).reshape(grid_size[1], grid_size[0])]}]} for col in carte_tirs_df.columns],
            'direction': 'down',
            'showactive': True,
            'x': -1, 
            'xanchor': 'left',
        }
    ]

    fig.layout = go.Layout(
        updatemenus=menus_de_mise_à_jour,
        title="Carte des tirs par rapport à la moyenne de la ligue par saison",
        title_x= 0.5,
    )

    largeur_img = image_patinoire.size[0]
    hauteur_img = image_patinoire.size[1]

    fig.update_xaxes(
        visible=True,
        range=[-42.5, 42.5],
        title_text='Distance depuis le centre de la patinoire (pieds)',
    )

    fig.update_yaxes(
        visible=True,
        range=[0, 100],
        scaleanchor="y",
        title_text='Distance depuis la ligne de but (pieds)',
    )

    # Ajouter l'image
    fig.add_layout_image(
            dict(
                source=image_patinoire,
                xref="x",
                yref="y",
                x=-42.5,
                sizex=largeur_img/5.5,
                y=100,
                sizey=hauteur_img/5.5,
                sizing="stretch",
                opacity=1,
                layer="below"
            )
    )

    fig.update_layout(
        autosize=False,
        width=largeur_img*1.5,
        height=hauteur_img*1.1
        )
    
    fig.write_html(save_fig)
    
    fig.show()

In [ ]:
import pandas as pd
from PIL import Image

# Load your DataFrame for the year 2016 !!!
# Make sure to adjust the file path according to your actual DataFrame location !!!!

# Load the rink image from the 'data' directory !!!!
rink_image_path = 'data/rink.png'  # Replace with the actual image file name !!!!
image_patinoire = Image.open(rink_image_path)

obtenir_coordonnées_côté_gauche(combined_df)
carte_tirs_df = préparer_données_de_cartedetir(combined_df)

générer_carte_tirs(carte_tirs_df, image_patinoire)

print("Shot map generated successfully.")

In [6]:
import numpy as np
import matplotlib.pyplot as plt

year = 2016
df = dataframes[year]

import numpy as np

def determine_shot_coords(row):
    """
    Determines shot coordinates based on the period and whether the team is home.
    Filters out shots made from the team's own half.

    Args:
        row (Series): A row from the DataFrame containing shot data.
        period (int): The current period (used to determine shooting direction).
    
    Returns:
        tuple: Adjusted (x, y) coordinates if the shot is valid, otherwise None.
    """

    try:
        
        x, y = eval(row['Coordinates'])
        if not isinstance(x, (int, float)) or not isinstance(y, (int, float)):
            return None
    except (SyntaxError, TypeError, NameError):
        return None

    is_home_team = row['Home']  
    period = row['Period']
    if period in [1, 3]:
        if is_home_team:
            if x > 0: 
                return None
        else:
            if x < 0:  
                return None
    elif period == 2:
        if is_home_team:
            if x < 0:  
                return None
        else:
            if x > 0:  
                return None
    x = abs(x)  
    return (y, x)


def adjust_shot_coordinates(df):
    """
    Adjusts shot coordinates based on the period and whether the team is home.
    Filters out shots from the team's own half of the rink.

    Args:
        df (DataFrame): DataFrame containing shot data.
    
    Returns:
        DataFrame: DataFrame with filtered and adjusted shot coordinates.
    """

    df['Adjusted Coordinates'] = df.apply(lambda row: determine_shot_coords(row), axis=1)
    df = df.dropna(subset=['Adjusted Coordinates'])
    return df

df = adjust_shot_coordinates(df)


def aggregate_shot_locations(df,factor=1, grid_size=1):
    y_bins = np.arange(0, 101 , grid_size)  
    x_bins = np.arange(-42.5, 43.5 , grid_size)  

    df['xCoord'], df['yCoord'] = zip(*df['Adjusted Coordinates'])
    shot_counts, _, _ = np.histogram2d(df['xCoord'], df['yCoord'], bins=[x_bins, y_bins])
    total_games = df['ID'].nunique()
    return shot_counts/total_games


league_average = aggregate_shot_locations(df,2)

In [7]:
def calculate_team_shot_rate(df, team_name):

    team_df = df[df['Team'] == team_name]
    team_shot_counts = aggregate_shot_locations(team_df)    
    return team_shot_counts


def calculate_shot_rate_difference(team_shot_rate, league_shot_rate, method='absolute'):

    if method == 'absolute':

        return team_shot_rate - league_shot_rate
    elif method == 'percentage':
        return (team_shot_rate - league_shot_rate) / league_shot_rate * 100
    else:
        raise ValueError("Method must be 'absolute' or 'percentage'.")

teams = df['Team'].unique()
rates={}
for team in teams:
    team_shot_rate = calculate_team_shot_rate(df, team)
    rates[team] = calculate_shot_rate_difference(team_shot_rate, league_average, method='absolute')


In [18]:
from scipy.ndimage import gaussian_filter
def générer_carte_tirs(shot_rate, image_patinoire, grid_size=(1, 1), echelle=100):
    """
    Produire une visualisation de la carte des tirs
    
    Paramètres :
        carte_tirs_df : sortie de la fonction préparer_carte_tirs()
        image_patinoire : image fournie de la patinoire de hockey
        grid_size : taille de la grille discrète pour laquelle nous calculons les densités de probabilité estimées par le noyau
    """
    shot_rate = gaussian_filter(shot_rate, sigma=3)
    image_patinoire = image_patinoire.crop((image_patinoire.size[0]/2, 0, image_patinoire.size[0], image_patinoire.size[1]))
    image_patinoire = image_patinoire.transpose(Image.Transpose.ROTATE_90)

    y = np.linspace(0, 89, grid_size[0])
    x = np.linspace(-42.5, 42.5, grid_size[1])
    fig = go.Figure()

    fig.add_trace(
        go.Contour(
            z=shot_rate.T,
            x=x,
            y=y,
            hoverongaps=False,
            opacity=0.8,
            zmin=-0.55, zmax=0.55,
            colorscale=[[0, '#0000FF'], [0.5, '#FFFFFF'], [1, '#FF0000']],
            colorbar=dict(title=f"Tirs excédentaires par heure par {echelle} pieds carrés", titleside="right")
        )
    )

    # menus_de_mise_à_jour = [
    #     {
    #         'buttons': 
    #            [{'method': 'restyle', 'label': col,'args': [{'z': [np.array(carte_tirs_df[col]).reshape(grid_size[1], grid_size[0])]}]} for col in carte_tirs_df.columns],
    #         'direction': 'down',
    #         'showactive': True,
    #         'x': -1, 
    #         'xanchor': 'left',
    #     }
    # ]

    fig.layout = go.Layout(
        # updatemenus=menus_de_mise_à_jour,
        title="Carte des tirs par rapport à la moyenne de la ligue par saison",
        title_x= 0.5,
    )

    largeur_img = image_patinoire.size[0]
    hauteur_img = image_patinoire.size[1]

    fig.update_xaxes(
        visible=True,
        range=[-42.5, 42.5],
        title_text='Distance depuis le centre de la patinoire (pieds)',
    )

    fig.update_yaxes(
        visible=True,
        range=[0, 100],
        scaleanchor="y",
        title_text='Distance depuis la ligne de but (pieds)',
    )

    # Ajouter l'image
    fig.add_layout_image(
            dict(
                source=image_patinoire,
                xref="x",
                yref="y",
                x=-42.5,
                sizex=largeur_img/5.5,
                y=100,
                sizey=hauteur_img/5.5,
                sizing="stretch",
                opacity=1,
                layer="below"
            )
    )

    fig.update_layout(
        autosize=False,
        width=largeur_img*1.5,
        height=hauteur_img*1.1
        )
    
    # fig.write_html(save_fig)
    
    fig.show()

In [19]:
rink_image_path = 'data/rink.png'  # Replace with the actual image file name !!!!
image_patinoire = Image.open(rink_image_path)
générer_carte_tirs(rates['Senators'],image_patinoire)